In [1]:
import pandas as pd
from pyspark.sql import SparkSession

In [2]:
# there is a bug, causing error during first call
try:
    from pyspark.ml.feature import Imputer
except:
    pass
from pyspark.ml.feature import Imputer

In [3]:
pd.read_csv("test1.csv")

,Name,Age,Experience
0,Milana Parks,30.0,7.0
1,Bodie Barton,22.0,12.0
2,Foster Weeks,45.0,3.0
3,Joshua Meyers,NaN,22.0
4,Rayna Ortiz,NaN,NaN
5,NaN,NaN,NaN
6,NaN,99.0,NaN
7,Paxton Velasquez,NaN,8.0


In [4]:
# use spark.stop() to stop it later
spark = SparkSession.builder.appName('ss1').getOrCreate()

In [5]:
spark

In [6]:
# dir(spark.read), help(spark.read)
# df_spark = spark.read.csv("test1.csv")
df_spark = spark.read.csv("test1.csv",header= True, inferSchema=True)

In [7]:
df_spark.show()

+----------------+----+----------+
|            Name| Age|Experience|
+----------------+----+----------+
|    Milana Parks|  30|         7|
|    Bodie Barton|  22|        12|
|    Foster Weeks|  45|         3|
|   Joshua Meyers|NULL|        22|
|     Rayna Ortiz|NULL|      NULL|
|            NULL|NULL|      NULL|
|            NULL|  99|      NULL|
|Paxton Velasquez|NULL|         8|
+----------------+----+----------+



In [8]:
df_spark.head(5)

[Row(Name='Milana Parks', Age=30, Experience=7),
 Row(Name='Bodie Barton', Age=22, Experience=12),
 Row(Name='Foster Weeks', Age=45, Experience=3),
 Row(Name='Joshua Meyers', Age=None, Experience=22),
 Row(Name='Rayna Ortiz', Age=None, Experience=None)]

In [9]:
df_spark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)



In [10]:
df_spark.columns

['Name', 'Age', 'Experience']

In [11]:
# df_spark.select("Name","Age")
df_spark[["Name", "Age"]].show()

+----------------+----+
|            Name| Age|
+----------------+----+
|    Milana Parks|  30|
|    Bodie Barton|  22|
|    Foster Weeks|  45|
|   Joshua Meyers|NULL|
|     Rayna Ortiz|NULL|
|            NULL|NULL|
|            NULL|  99|
|Paxton Velasquez|NULL|
+----------------+----+



In [12]:
df_spark.summary().show()

+-------+------------+-----------------+-----------------+
|summary|        Name|              Age|       Experience|
+-------+------------+-----------------+-----------------+
|  count|           6|                4|                5|
|   mean|        NULL|             49.0|             10.4|
| stddev|        NULL|34.66987164671943|7.231873892705818|
|    min|Bodie Barton|               22|                3|
|    25%|        NULL|               22|                7|
|    50%|        NULL|               30|                8|
|    75%|        NULL|               45|               12|
|    max| Rayna Ortiz|               99|               22|
+-------+------------+-----------------+-----------------+



In [13]:
df_spark.describe().show()

+-------+------------+-----------------+-----------------+
|summary|        Name|              Age|       Experience|
+-------+------------+-----------------+-----------------+
|  count|           6|                4|                5|
|   mean|        NULL|             49.0|             10.4|
| stddev|        NULL|34.66987164671943|7.231873892705818|
|    min|Bodie Barton|               22|                3|
|    max| Rayna Ortiz|               99|               22|
+-------+------------+-----------------+-----------------+



In [14]:
df_spark.dtypes

[('Name', 'string'), ('Age', 'int'), ('Experience', 'int')]

In [15]:
# create new column, returns a copy
df_spark.withColumn("Exp_After_2_Years",df_spark["Experience"]+2).show()

+----------------+----+----------+-----------------+
|            Name| Age|Experience|Exp_After_2_Years|
+----------------+----+----------+-----------------+
|    Milana Parks|  30|         7|                9|
|    Bodie Barton|  22|        12|               14|
|    Foster Weeks|  45|         3|                5|
|   Joshua Meyers|NULL|        22|               24|
|     Rayna Ortiz|NULL|      NULL|             NULL|
|            NULL|NULL|      NULL|             NULL|
|            NULL|  99|      NULL|             NULL|
|Paxton Velasquez|NULL|         8|               10|
+----------------+----+----------+-----------------+



In [16]:
# Renaming Columns, returns a copy
df_spark.withColumnsRenamed({'Name': 'Full_Name', 'Experience': 'Exprnc'}).show()

+----------------+----+------+
|       Full_Name| Age|Exprnc|
+----------------+----+------+
|    Milana Parks|  30|     7|
|    Bodie Barton|  22|    12|
|    Foster Weeks|  45|     3|
|   Joshua Meyers|NULL|    22|
|     Rayna Ortiz|NULL|  NULL|
|            NULL|NULL|  NULL|
|            NULL|  99|  NULL|
|Paxton Velasquez|NULL|     8|
+----------------+----+------+



In [17]:
# drop NaN values
df_spark.na.drop().show()

+------------+---+----------+
|        Name|Age|Experience|
+------------+---+----------+
|Milana Parks| 30|         7|
|Bodie Barton| 22|        12|
|Foster Weeks| 45|         3|
+------------+---+----------+



In [18]:
df_spark.na.drop(how="all").show()

+----------------+----+----------+
|            Name| Age|Experience|
+----------------+----+----------+
|    Milana Parks|  30|         7|
|    Bodie Barton|  22|        12|
|    Foster Weeks|  45|         3|
|   Joshua Meyers|NULL|        22|
|     Rayna Ortiz|NULL|      NULL|
|            NULL|  99|      NULL|
|Paxton Velasquez|NULL|         8|
+----------------+----+----------+



In [19]:
# defullt: how="any"
df_spark.na.drop(how="all").show()

+----------------+----+----------+
|            Name| Age|Experience|
+----------------+----+----------+
|    Milana Parks|  30|         7|
|    Bodie Barton|  22|        12|
|    Foster Weeks|  45|         3|
|   Joshua Meyers|NULL|        22|
|     Rayna Ortiz|NULL|      NULL|
|            NULL|  99|      NULL|
|Paxton Velasquez|NULL|         8|
+----------------+----+----------+



In [20]:
# drop rows that don't have at least #thresh non-null values
# overwrites how
df_spark.na.drop(thresh=2).show()

+----------------+----+----------+
|            Name| Age|Experience|
+----------------+----+----------+
|    Milana Parks|  30|         7|
|    Bodie Barton|  22|        12|
|    Foster Weeks|  45|         3|
|   Joshua Meyers|NULL|        22|
|Paxton Velasquez|NULL|         8|
+----------------+----+----------+



In [21]:
df_spark.na.drop(how="all",subset=["Name","Experience"]).show()

+----------------+----+----------+
|            Name| Age|Experience|
+----------------+----+----------+
|    Milana Parks|  30|         7|
|    Bodie Barton|  22|        12|
|    Foster Weeks|  45|         3|
|   Joshua Meyers|NULL|        22|
|     Rayna Ortiz|NULL|      NULL|
|Paxton Velasquez|NULL|         8|
+----------------+----+----------+



In [22]:
# subsets must be of similar types
df_spark.na.fill(0, subset=["Age", "Experience"]).show()

+----------------+---+----------+
|            Name|Age|Experience|
+----------------+---+----------+
|    Milana Parks| 30|         7|
|    Bodie Barton| 22|        12|
|    Foster Weeks| 45|         3|
|   Joshua Meyers|  0|        22|
|     Rayna Ortiz|  0|         0|
|            NULL|  0|         0|
|            NULL| 99|         0|
|Paxton Velasquez|  0|         8|
+----------------+---+----------+



In [23]:
df_spark.na.fill({"name": "Unknown", "age": 0}).show()

+----------------+---+----------+
|            Name|Age|Experience|
+----------------+---+----------+
|    Milana Parks| 30|         7|
|    Bodie Barton| 22|        12|
|    Foster Weeks| 45|         3|
|   Joshua Meyers|  0|        22|
|     Rayna Ortiz|  0|      NULL|
|         Unknown|  0|      NULL|
|         Unknown| 99|      NULL|
|Paxton Velasquez|  0|         8|
+----------------+---+----------+



In [24]:
columns = ["Age", "Experience"]
imputer = Imputer(inputCols=columns, outputCols=[f"{i}_" for i in columns])
imputer.setStrategy("mean").fit(df_spark).transform(df_spark).show()

+----------------+----+----------+----+-----------+
|            Name| Age|Experience|Age_|Experience_|
+----------------+----+----------+----+-----------+
|    Milana Parks|  30|         7|  30|          7|
|    Bodie Barton|  22|        12|  22|         12|
|    Foster Weeks|  45|         3|  45|          3|
|   Joshua Meyers|NULL|        22|  49|         22|
|     Rayna Ortiz|NULL|      NULL|  49|         10|
|            NULL|NULL|      NULL|  49|         10|
|            NULL|  99|      NULL|  99|         10|
|Paxton Velasquez|NULL|         8|  49|          8|
+----------------+----+----------+----+-----------+



In [25]:
spark.stop()